# установка библиотек

In [1]:
#!/usr/bin/env python3
import subprocess
import sys

def install_dependencies():
    
    dependencies = [
        "PyPDF2",
        "python-docx", 
        "pdfminer.six",
        "ebooklib",
        "beautifulsoup4",
        "html2text",
        "pandas",
        "openpyxl",
        "lxml",
        "chardet",
        "python-magic"
    ]
    
    print("Установка зависимостей для универсального парсера файлов...")
    print("=" * 60)
    
    for package in dependencies:
        try:
            print(f"Устанавливаю {package}...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
            print(f"{package} успешно установлен")
        except subprocess.CalledProcessError as e:
            print(f"✗ Ошибка при установке {package}: {e}")
        print("-" * 40)
    
    print("\nВсе зависимости установлены!")
    print("Можно запускать парсер: python universal_parser.py")

if __name__ == "__main__":
    install_dependencies()


Установка зависимостей для универсального парсера файлов...
Устанавливаю PyPDF2...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.7 MB/s eta 0:00:00
PyPDF2 успешно установлен
----------------------------------------
Устанавливаю python-docx...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 5.6 MB/s eta 0:00:00
python-docx успешно установлен
----------------------------------------
Устанавливаю pdfminer.six...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 47.7 MB/s eta 0:00:00
pdfminer.six успешно установлен
----------------------------------------
Устанавливаю ebooklib...
ebooklib успешно установлен
----------------------------------------
Устанавливаю beautifulsoup4...
beautifulsoup4 успешно установлен
----------------------------------------
Устанавливаю html2text...
html2text успешно установлен
----------------------------------------
Устанавливаю pandas...
pandas успешно установлен
----------------------------------------
Устанавливаю openpyxl...


# Парсер доков

In [2]:
import os
import re
import magic
import chardet
from typing import Dict, Any, List
import PyPDF2
from docx import Document
import pdfminer.high_level
from ebooklib import epub
from bs4 import BeautifulSoup
import pandas as pd

clean_text = ""

class TextProcessor:
    @staticmethod
    def clean_text(text: str) -> str:
        text = re.sub(r'\s+', ' ', text)
        return text.strip()
    
    @staticmethod
    def split_into_sentences(text: str) -> List[str]:
        sentences = re.split(r'[.!?]+', text)
        return [s.strip() for s in sentences if s.strip()]
    
    @staticmethod
    def get_word_frequency(text: str, top_n: int = 10) -> Dict[str, int]:
        words = text.lower().split()
        word_count = {}
        for word in words:
            word = re.sub(r'[^\w]', '', word)
            if word and len(word) > 2:
                word_count[word] = word_count.get(word, 0) + 1
        return dict(sorted(word_count.items(), key=lambda x: x[1], reverse=True)[:top_n])
    
    @staticmethod
    def detect_encoding(file_path: str) -> str:
        with open(file_path, 'rb') as file:
            raw_data = file.read(10000)
            result = chardet.detect(raw_data)
            return result['encoding'] or 'utf-8'

class UniversalParser:
    def __init__(self):
        self.supported_formats = {
            '.txt': self.parse_txt,
            '.pdf': self.parse_pdf,
            '.docx': self.parse_docx,
            '.epub': self.parse_epub,
            '.html': self.parse_html,
            '.htm': self.parse_html,
            '.csv': self.parse_csv,
            '.xlsx': self.parse_excel,
            '.xls': self.parse_excel,
        }
        self.text_processor = TextProcessor()
    
    def get_file_extension(self, file_path: str) -> str:
        return os.path.splitext(file_path)[1].lower()
    
    def detect_file_type(self, file_path: str) -> str:
        try:
            return magic.from_file(file_path, mime=True)
        except:
            return "unknown"
    
    def is_supported(self, file_path: str) -> bool:
        ext = self.get_file_extension(file_path)
        return ext in self.supported_formats
    
    def get_supported_formats(self) -> List[str]:
        return list(self.supported_formats.keys())
    
    def parse_txt(self, file_path: str) -> Dict[str, Any]:
        try:
            encoding = self.text_processor.detect_encoding(file_path)
            with open(file_path, 'r', encoding=encoding, errors='ignore') as file:
                content = file.read()
            cleaned_content = self.text_processor.clean_text(content)
            return {
                'content': cleaned_content,
                'metadata': {'encoding': encoding, 'lines': len(content.split('\n')), 'words': len(content.split()), 'characters': len(content)}
            }
        except Exception as e:
            raise Exception(f"TXT error: {str(e)}")
    
    def parse_pdf(self, file_path: str) -> Dict[str, Any]:
        try:
            text_content = ""
            metadata = {}
            with open(file_path, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                if hasattr(pdf_reader, 'metadata') and pdf_reader.metadata:
                    metadata = {
                        'title': str(pdf_reader.metadata.get('/Title', '')),
                        'author': str(pdf_reader.metadata.get('/Author', '')),
                        'pages': len(pdf_reader.pages)
                    }
                for page in pdf_reader.pages:
                    if hasattr(page, 'extract_text'):
                        text_content += page.extract_text() + "\n"
            if not text_content.strip():
                text_content = pdfminer.high_level.extract_text(file_path)
            cleaned_content = self.text_processor.clean_text(text_content)
            return {'content': cleaned_content, 'metadata': metadata}
        except Exception as e:
            raise Exception(f"PDF error: {str(e)}")
    
    def parse_docx(self, file_path: str) -> Dict[str, Any]:
        try:
            doc = Document(file_path)
            content = []
            for paragraph in doc.paragraphs:
                if paragraph.text.strip():
                    content.append(paragraph.text)
            for table in doc.tables:
                for row in table.rows:
                    for cell in row.cells:
                        if cell.text.strip():
                            content.append(cell.text)
            full_text = "\n".join(content)
            cleaned_content = self.text_processor.clean_text(full_text)
            core_properties = doc.core_properties
            return {
                'content': cleaned_content,
                'metadata': {
                    'title': core_properties.title,
                    'author': core_properties.author,
                    'paragraphs': len(doc.paragraphs),
                    'tables': len(doc.tables)
                }
            }
        except Exception as e:
            raise Exception(f"DOCX error: {str(e)}")
    
    def parse_epub(self, file_path: str) -> Dict[str, Any]:
        try:
            book = epub.read_epub(file_path)
            content = []
            for item in book.get_items():
                if item.get_type() == epub.ITEM_DOCUMENT:
                    soup = BeautifulSoup(item.get_content(), 'html.parser')
                    text = soup.get_text(separator='\n')
                    if text.strip():
                        content.append(text)
            full_text = "\n\n".join(content)
            cleaned_content = self.text_processor.clean_text(full_text)
            metadata = {}
            if book.get_metadata('DC', 'title'):
                metadata['title'] = book.get_metadata('DC', 'title')[0][0]
            if book.get_metadata('DC', 'creator'):
                metadata['author'] = book.get_metadata('DC', 'creator')[0][0]
            return {'content': cleaned_content, 'metadata': metadata}
        except Exception as e:
            raise Exception(f"EPUB error: {str(e)}")
    
    def parse_html(self, file_path: str) -> Dict[str, Any]:
        try:
            encoding = self.text_processor.detect_encoding(file_path)
            with open(file_path, 'r', encoding=encoding, errors='ignore') as file:
                html_content = file.read()
            soup = BeautifulSoup(html_content, 'html.parser')
            for script in soup(["script", "style", "nav", "footer", "header"]):
                script.decompose()
            text = soup.get_text(separator='\n')
            lines = (line.strip() for line in text.splitlines())
            chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
            cleaned_text = '\n'.join(chunk for chunk in chunks if chunk)
            links = [a.get('href') for a in soup.find_all('a', href=True)]
            return {
                'content': cleaned_text,
                'metadata': {
                    'title': soup.title.string if soup.title else '',
                    'links_count': len(links),
                    'encoding': encoding
                }
            }
        except Exception as e:
            raise Exception(f"HTML error: {str(e)}")
    
    def parse_csv(self, file_path: str) -> Dict[str, Any]:
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                first_line = file.readline()
            if ';' in first_line:
                delimiter = ';'
            elif ',' in first_line:
                delimiter = ','
            elif '\t' in first_line:
                delimiter = '\t'
            else:
                delimiter = ','
            df = pd.read_csv(file_path, delimiter=delimiter, encoding='utf-8', on_bad_lines='skip')
            text_content = df.to_string()
            return {
                'content': text_content,
                'metadata': {
                    'rows': len(df),
                    'columns': len(df.columns),
                    'columns_names': list(df.columns),
                    'delimiter': delimiter
                }
            }
        except Exception as e:
            raise Exception(f"CSV error: {str(e)}")
    
    def parse_excel(self, file_path: str) -> Dict[str, Any]:
        try:
            excel_file = pd.ExcelFile(file_path)
            sheets_data = {}
            sheets_info = []
            for sheet_name in excel_file.sheet_names:
                df = pd.read_excel(file_path, sheet_name=sheet_name)
                sheet_content = df.to_string()
                sheets_data[sheet_name] = sheet_content
                sheets_info.append({'name': sheet_name, 'rows': len(df), 'columns': len(df.columns)})
            full_content = "\n\n".join([f"=== {name} ===\n{content}" for name, content in sheets_data.items()])
            return {
                'content': full_content,
                'metadata': {
                    'sheets_count': len(excel_file.sheet_names),
                    'sheets_info': sheets_info,
                    'sheet_names': excel_file.sheet_names
                }
            }
        except Exception as e:
            raise Exception(f"Excel error: {str(e)}")
    
    def parse_file(self, file_path: str) -> Dict[str, Any]:
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"File not found: {file_path}")
        if not os.path.isfile(file_path):
            raise ValueError(f"Not a file: {file_path}")
        ext = self.get_file_extension(file_path)
        if not self.is_supported(file_path):
            detected_type = self.detect_file_type(file_path)
            raise ValueError(f"Unsupported format: {ext} (detected: {detected_type})")
        parser_func = self.supported_formats[ext]
        return parser_func(file_path)
    
    def extract_text_only(self, file_path: str) -> str:
        result = self.parse_file(file_path)
        return result['content']
    
    def get_statistics(self, file_path: str) -> Dict[str, Any]:
        result = self.parse_file(file_path)
        content = result['content']
        stats = {
            'file_name': os.path.basename(file_path),
            'file_size': os.path.getsize(file_path),
            'file_format': self.get_file_extension(file_path),
            'detected_type': self.detect_file_type(file_path),
            'lines': len(content.split('\n')),
            'words': len(content.split()),
            'characters': len(content),
            'unique_words': len(set(content.split())),
            'sentences': len(self.text_processor.split_into_sentences(content))
        }
        word_freq = self.text_processor.get_word_frequency(content, 5)
        stats['top_words'] = word_freq
        if 'metadata' in result:
            stats.update({'file_' + k: v for k, v in result['metadata'].items()})
        return stats
    
    def search_in_file(self, file_path: str, search_term: str, case_sensitive: bool = False) -> List[Dict]:
        content = self.extract_text_only(file_path)
        if not case_sensitive:
            content_lower = content.lower()
            search_term_lower = search_term.lower()
        else:
            content_lower = content
            search_term_lower = search_term
        lines = content.split('\n')
        results = []
        for i, line in enumerate(lines, 1):
            if case_sensitive:
                search_content = line
            else:
                search_content = line.lower()
            if search_term_lower in search_content:
                start = 0
                while True:
                    pos = search_content.find(search_term_lower, start)
                    if pos == -1:
                        break
                    results.append({
                        'line_number': i,
                        'position': pos + 1,
                        'context': line[max(0, pos-20):pos+len(search_term)+20],
                        'match': line[pos:pos+len(search_term)]
                    })
                    start = pos + 1
        return results

def main():
    global clean_text
    parser = UniversalParser()
    print("Supported formats:", ", ".join(parser.get_supported_formats()))
    
    file_path = input("Enter file path: ").strip().strip('"')
    
    try:
        if parser.is_supported(file_path):
            result = parser.parse_file(file_path)
            clean_text = result['content']
            print(clean_text)

        else:
            print("Unsupported file format")
            
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()


Supported formats: .txt, .pdf, .docx, .epub, .html, .htm, .csv, .xlsx, .xls


Enter file path:  /kaggle/input/oleg-dorkanov-resume/Oleg Drokanov CV EN.pdf


Oleg Drokanov DoB 12.03.2009 , Saint Petersburg , Russia +79214054678 | oleg.drokanov@yandex.ru | https: //github.com/olegg366 Key Information I am developer and researcher in the field of artificial intelligence, machine learning, and robotics. I enjoy solving complex problems and turning ideas into working projects —from experimenting with neural networks to buil ding real robotic systems. I have successful experience in international olympiads and practical experience in implementing full -scale AI solutions. Achievements and Awards • Gold Medal | IOAI 2025 (International Olympiad in Artificial Intelligence) • 1st Place | International AI Challenge, "Education" case (2024) • Prize Winner | DANO, Data Analysis Olympiad (2024) • Prize Winner | All-Russian School Olympiad in Technology, "Robotics" track (2025) • 4th Place | NEOAI (Northern Eurasia Olympiad in Artificial Intelligence, 2025) • 2nd Place | Russian Robotics Olympiad, "Creative Category . AI", project: "AI Artist Robot" (20

In [3]:
!pip install -q ollama transformers accelerate sentencepiece protobuf lsprotocol ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.2/76.2 kB 1.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.0/70.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.5 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%                                                    8.8%###########################                            65.1%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [5]:
import warnings

warnings.filterwarnings("ignore")

In [44]:
# качаем оламу
import subprocess
import time
import threading

def run_ollama():
    subprocess.run(["ollama", "serve"])

# Запускаем в отдельном потоке
ollama_thread = threading.Thread(target=run_ollama, daemon=True)
ollama_thread.start()

# Даем время на запуск
time.sleep(5)

!ollama pull llama3.1:8b-instruct-q4_0

Error: listen tcp 127.0.0.1:11434: bind: address already in use


[GIN] 2025/09/29 - 14:16:38 | 200 |      33.729µs |       127.0.0.1 | HEAD     "/"
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ [GIN] 2025/09/29 - 14:16:38 | 200 |  351.462058ms |       127.0.0.1 | POST     "/api/pull"
pulling manifest 
pulling 8eeb52dfb3bb: 100% ▕██████████████████▏ 4.7 GB                         
pulling 948af2743fc7: 100% ▕██████████████████▏ 1.5 KB                         
pulling 0ba8f0e314b4: 100% ▕██████████████████▏  12 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 1a4c3c319823: 100% ▕██████████████████▏  485 B                         
verifying sha256 digest 
writing manifest 
success 


# Analyze Agent

In [37]:
pr_aa = '''Ты — «Analyze Agent», эксперт по анализу литературного и медийного контента. Твоя задача — тщательно анализировать предоставленный текст и выделять из него ключевую информацию по заданным критериям.

**ИНСТРУКЦИИ:**
1.  Внимательно прочитай весь текст.
2.  На основе содержания текста заполни каждый пункт приведенной ниже структуры.
3.  Будь точен и объективен. Если какой-то пункт невозможно определить, укажи "Неясно" и кратко объясни почему.
4.  Отвечай **ТОЛЬКО** в формате JSON, строго следуя приведенной структуре.

**ВЫХОДНАЯ СТРУКТУРА (JSON):**
{
  "scenes": ["Краткое описание основной сцены 1", "Краткое описание основной сцены 2", ...],
  "characters": ["Имя или роль персонажа 1", "Имя или роль персонажа 2", ...],
  "tone": "Общий тон повествования (например, мрачный, радостный, саркастический, напряженный)",
  "target_audience": "Описание целевой аудитории текста (например, фанаты научной фантастики, молодые родители, студенты-историки)",
  "age_rating": "Рекомендуемая возрастная категория (G, PG, PG-13, R, etc. или по российской системе: 0+, 6+, 12+, 16+, 18+)",
  "style": "Преобладающий стиль (юмор, серьезный, образовательный, сатирический, драматический - можно указать несколько через запятую)",
  "key_quotes": ["Цитата 1", "Цитата 2", ...] // Выбери 3-5 самых ярких или значимых цитат.
}

**ПРИМЕР ОТВЕТА:** 
**отвечай только json файлом, ничего лишнего**
{
  "scenes": ["Диалог в кафе между двумя шпионами", "Погоня по крышам ночного города"],
  "characters": ["Агент Х", "Агент Y", "Таинственный незнакомец"],
  "tone": "Напряженный и полный сарказма",
  "target_audience": "Взрослые любители шпионских триллеров",
  "age_rating": "PG-13 (16+)",
  "style": "серьезный, драматический",
  "key_quotes": ["«Холодная война закончилась, но мы все еще в тени», — сказал Агент Х, помешивая кофе.", "«На крышах нет правил, есть только гравитация», — крикнул Агент Y, прыгая через пропасть."]
}

Начинай анализ сразу после получения текста.

вот текст для обработки: ''' + f'{clean_text}'

In [48]:
import requests
import sys

def ask_ollama(question: str) -> str:
    api_url = "http://localhost:11434/api/generate"
    payload = {
        "model": "llama3.1:8b-instruct-q4_0",
        "prompt": question,
        "stream": False,
        "options": {
            "temperature": 0.7,
            "num_predict": 2000
        }
    }
    
    try:
        response = requests.post(api_url, json=payload, timeout=500)
        response.raise_for_status()
        return response.json()["response"]
    except requests.exceptions.RequestException as e:
        return f"Ошибка запроса: {str(e)}"

def check_ollama():
    """Проверяет доступность Ollama"""
    try:
        response = requests.get("http://localhost:11434/api/tags", timeout=10)
        if response.status_code == 200:
            print("✅ Ollama работает корректно")
            return True
        else:
            print("❌ Ollama не отвечает")
            return False
    except:
        print("❌ Не удалось подключиться к Ollama")
        return False

if __name__ == "__main__":
    
    question = pr_aa
    
    if check_ollama():
        answer_aa = ask_ollama(question)
        print(answer_aa)

[GIN] 2025/09/29 - 14:18:40 | 200 |     941.647µs |       127.0.0.1 | GET      "/api/tags"
✅ Ollama работает корректно
[GIN] 2025/09/29 - 14:18:48 | 200 |  7.170009611s |       127.0.0.1 | POST     "/api/generate"
{
  "scenes": ["Сценка описания работы в области ИИ и ML", "Погоня за достижениями в международных олимпиадах"],
  "characters": ["Олег Дроканов - разработчик и исследователь в области ИИ, ML и робототехники"],
  "tone": "Упругий и профессиональный",
  "target_audience": "Профессионалы в области ИИ, ML и робототехники, студенты-специалисты по компьютерным наукам",
  "age_rating": "Не применимо",
  "style": "Образовательный, технический",
  "key_quotes": ["«Я люблю решать сложные проблемы и преобразовывать идеи в рабочие проекты», — говорит Олег.", "«Моя цель - не просто обучить модели, но сделать их способными принимать решения в реальных условиях»."]
}


# Script Writer (LLM)

In [49]:
pr_sw = '''Ты — «Script Writer», профессиональный сценарист и специалист по prompt engineering для генерации изображений. Твоя задача — адаптировать литературный текст в формат сценария из серии панелей (как в комиксе или раскадровке).

**ИНСТРУКЦИИ:**
1.  Разбей исходный текст на логические сцены или ключевые моменты. Каждый момент станет отдельной "панелью".
2.  Для каждой панели создай:
    *   **Номер панели:** (Panel 1, Panel 2...)
    *   **Визуальное описание:** Яркое, детализированное описание того, что должно быть на изображении. Включи информацию о персонажах (внешность, эмоции, поза), окружении, освещении, ключевых объектах и стиле.
    *   **Диалог/Текст:** Реплики персонажей или повествовательный текст, который будет отображаться на панели (в "пузыре" или как подпись). Если диалога нет, можно оставить пустым или добавить звуко-ономатопею (например, *БАМ!*).
    *   **Image Prompt:** Готовый, оптимизированный промпт для AI-генератора изображений (например, Stable Diffusion, Midjourney). Промпт должен быть на английском языке для лучшей совместимости, детализированным и включать ключевые слова по стилю (e.g., `cinematic shot, photorealistic, dynamic lighting, detailed background`).
3.  Стремись к визуальной насыщенности. "Показывай, а не рассказывай".
4.  Отвечай **ТОЛЬКО** в формате JSON, строго следуя приведенной структуре.

**ВЫХОДНАЯ СТРУКТУРА (JSON):**
{
  "script_title": "Название сценария на основе текста",
  "panels": [
    {
      "panel_number": 1,
      "visual_description": "Детальное описание сцены на русском.",
      "dialogue_text": "Реплики или текст для этой панели.",
      "image_prompt": "A detailed, cinematic image of [описание сцены на английском]. Style: [указание стиля]. Lighting: [описание освещения]. --ar 16:9 --v 6.0"
    },
    // ... последующие панели
  ]
}

**ПРИМЕР ОТВЕТА:** 
**отвечай только json файлом, ничего лишнего**
{
  "script_title": "Встреча в кафе",
  "panels": [
    {
      "panel_number": 1,
      "visual_description": "Двое мужчин в дорогих костюмах сидят за маленьким столиком в затемненном кафе. Один (Агент Х) помешивает ложечкой эспрессо, его лицо освещено лишь светом от неоновой вывески за окном. Он смотрит на собеседника с легкой усмешкой. Второй (Агент Y) напряжен, его взгляд направлен в сторону.",
      "dialogue_text": "Агент Х: «Холодная война закончилась, но мы все еще в тени».",
      "image_prompt": "A cinematic shot of two well-dressed spies in a moody, dark cafe at night. One man stirs his espresso with a sly smirk, neon light from a window sign illuminates his face. The other man looks tense, glancing sideways. Photorealistic, film noir style, dramatic lighting, shallow depth of field. --ar 16:9 --v 6.0"
    }
    // ... Panel 2 и т.д.
  ]
}

Начинай создание сценария сразу после получения текста.

текст для обработки:''' + f'{clean_text}'

In [50]:
def check_ollama():
    """Проверяет доступность Ollama"""
    try:
        response = requests.get("http://localhost:11434/api/tags", timeout=10)
        if response.status_code == 200:
            print("✅ Ollama работает корректно")
            return True
        else:
            print("❌ Ollama не отвечает")
            return False
    except:
        print("❌ Не удалось подключиться к Ollama")
        return False

if __name__ == "__main__":
    
    question = pr_sw
    
    if check_ollama():
        answer_sw = ask_ollama(question)
        print(answer_sw)

[GIN] 2025/09/29 - 14:19:00 | 200 |     937.081µs |       127.0.0.1 | GET      "/api/tags"
✅ Ollama работает корректно
[GIN] 2025/09/29 - 14:19:22 | 200 | 22.497981099s |       127.0.0.1 | POST     "/api/generate"
**Сценарий с использованием панелей**


{
  "script_title": "Портфолио разработчика",
  "panels": [
    {
      "panel_number": 1,
      "visual_description": "Молодой парень (Oleg Drokanov) в форменной одежде стоит перед фоном с его логотипом и информацией об аккаунте GitHub. Он смотрит прямо на камеру, улыбается. На заднем плане видны несколько компьютеров и экраны с кодом.",
      "dialogue_text": "",
      "image_prompt": "A young professional (Oleg Drokanov) stands in front of a background with his GitHub logo and account information. He smiles directly at the camera, wearing formal attire. Multiple computers and screens with code are visible on the background. Photorealistic, cinematic style, shallow depth of field. --ar 16:9 --v 6.0"
    },
    {
      "panel_number": 